In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.metrics import MeanSquaredError as mse

In [79]:
# Caricare il modello
model = load_model('my_model.keras', custom_objects={'mse':mse}, compile=False)

In [71]:
# Funzione per ottenere lo stato attuale della stanza
def get_current_state():
    #n_people = get_people()
    n_people = 4
    ext_temperature = data['ext_temperature'].tail(5).mean()
    temperature_now = data['temperature_now'].tail(5).mean()
    pressure_now = data['pressure_now'].tail(5).mean()
    brightness_now = data['brightness_now'].tail(5).mean()
    humidity_now = data['humidity_now'].tail(5).mean()
    co2_now = data['co2_now'].tail(5).mean()
    window1_now, window2_now, window3_now, window4_now = np.array(data.tail(1)['window1_now'])[0], np.array(data.tail(1)['window2_now'])[0], np.array(data.tail(1)['window3_now'])[0], np.array(data.tail(1)['window4_now'])[0]
    shutter1_now, shutter2_now, shutter3_now, shutter4_now = np.array(data.tail(1)['shutter1_now'])[0], np.array(data.tail(1)['shutter2_now'])[0], np.array(data.tail(1)['shutter3_now'])[0], np.array(data.tail(1)['shutter4_now'])[0]
    date = np.array(data.tail(1)['date'])[0]
    time = np.array(data.tail(1)['time'])[0]
    room_size = np.array(data.tail(1)['room_size'])[0]
    
    return {
        'n_people': n_people, 'room_size': room_size, 'date': date, 'time':time, 'ext_temperature': ext_temperature,
        'temperature_now': temperature_now, 'co2_now': co2_now, 'pressure_now': pressure_now, 'brightness_now': brightness_now, 'humidity_now': humidity_now, 
        'window1_now': window1_now, 'window2_now': window2_now, 'window3_now': window3_now, 'window4_now': window4_now,
        'shutter1_now': shutter1_now, 'shutter2_now': shutter2_now, 'shutter3_now': shutter3_now, 'shutter4_now': shutter4_now
    }


# Funzione per ottenere i valori target definiti dall'utente
def get_opt():
    return {
        'temperature_opt': 22, 'co2_opt': 350, 'pressure_opt': 1.20, 'brightness_opt': 4.30, 'humidity_opt': 30
    }

In [50]:
# Preprocessing dei dati come nel notebook di training
def preprocess_data(data, input_values):
    # Normalize numerical features
    numerical_features = data.drop(columns=['window1_tg', 'window2_tg', 'window3_tg', 'window4_tg', 'shutter1_tg', 'shutter2_tg', 'shutter3_tg', 'shutter4_tg']).select_dtypes(include=[np.number]).columns.tolist()
    current_state_df = pd.DataFrame(data, columns=numerical_features, index=[0])
    current_state_df[numerical_features] = (current_state_df[numerical_features] - data[numerical_features].mean()) / data[numerical_features].std()

    # Convert time to minutes from midnight and extract month from date
    current_state_df['time'] = np.array(pd.to_datetime(data['time'].tail(1), format='%H:%M:%S').dt.hour * 60 + pd.to_datetime(data['time'].tail(1), format='%H:%M:%S').dt.minute)[0]
    current_state_df['date'] = np.array(pd.to_datetime(data['date'].tail(1)).dt.month)[0]
    
    new_order = ['n_people', 'room_size', 'date', 'time', 'ext_temperature', 'temperature_now', 'co2_now', 'pressure_now', 'brightness_now', 'humidity_now', 'temperature_opt', 'co2_opt', 'pressure_opt', 'brightness_opt', 'humidity_opt', 'window1_now', 'window2_now', 'window3_now', 'window4_now', 'shutter1_now', 'shutter2_now', 'shutter3_now', 'shutter4_now']
    current_state_df = current_state_df[new_order]
    
    # Encode categorical features
    categorical_features = ['date']
    current_state_df = pd.get_dummies(current_state_df, columns=categorical_features)

    return current_state_df

In [80]:
# Caricare il dataset per preprocessing
data = pd.read_csv('final_dataset.csv')
new_order = ['n_people',  'room_size', 'date', 'time', 'ext_temperature', 'temperature_now', 'co2_now', 'pressure_now', 'brightness_now', 'humidity_now', 'temperature_opt', 'co2_opt', 'pressure_opt', 'brightness_opt', 'humidity_opt', 'window1_now', 'window2_now', 'window3_now', 'window4_now', 'shutter1_now', 'shutter2_now', 'shutter3_now', 'shutter4_now', 'window1_tg', 'window2_tg', 'window3_tg', 'window4_tg', 'shutter1_tg', 'shutter2_tg', 'shutter3_tg', 'shutter4_tg']
data = data[new_order]

In [72]:
# Ottenere lo stato attuale preprocessato
current_state_values = get_current_state()
optimal_values = get_opt()
input_values = {**current_state_values, **optimal_values}

In [76]:
# Preprocessare i dati
input_values_preprocessed = preprocess_data(data, input_values)
print(input_values_preprocessed)

   n_people  room_size  time  ext_temperature  temperature_now  co2_now  \
0 -1.323934        NaN   890        -0.754499         0.760505 -1.02021   

   pressure_now  brightness_now  humidity_now  temperature_opt  ...  \
0      1.551957        -1.04254      0.427171        -0.107037  ...   

   humidity_opt  window1_now  window2_now  window3_now  window4_now  \
0      1.300195    -0.348501    -2.112663    -0.363359    -0.378498   

   shutter1_now  shutter2_now  shutter3_now  shutter4_now  date_6  
0       0.75929      0.869412      0.898966      0.827388    True  

[1 rows x 23 columns]


In [77]:
# Fare una previsione con il modello
prediction = model.predict(input_values_preprocessed)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


In [78]:
print(prediction)

[[-0.00610232  0.31217998  0.02215894 -0.09445719  0.29743284  0.34070414
   0.31779224  0.247432  ]]


In [81]:
def denormalizer(data, prediction):
    # Denormalize the prediction
    prediction_denormalized = prediction.copy()
    prediction_denormalized = prediction_denormalized * data[['window1_tg', 'window2_tg', 'window3_tg', 'window4_tg', 'shutter1_tg', 'shutter2_tg', 'shutter3_tg', 'shutter4_tg']].std() + data[['window1_tg', 'window2_tg', 'window3_tg', 'window4_tg', 'shutter1_tg', 'shutter2_tg', 'shutter3_tg', 'shutter4_tg']].mean()
    
    return prediction_denormalized

In [82]:
# Denormalizzare la previsione
prediction_denormalized = denormalizer(data, prediction[0])

In [83]:
prediction_denormalized = np.array(prediction_denormalized)
prediction_denormalized

array([0.04147424, 1.01716447, 0.06099432, 0.03733467, 0.89722966,
       0.93903917, 0.92786573, 0.86148043])

In [63]:
# Assume prediction_denormalized is already defined and has at least 8 elements
prediction_denormalized[:4] = [int(float(1 if x >= 0.5 else 0)) for x in prediction_denormalized[:4]]
prediction_denormalized[4:8] = [round(x, 1) for x in prediction_denormalized[4:8]]

In [64]:
print(prediction_denormalized)

[0.  1.  0.  0.  0.9 0.9 0.9 0.9]


In [65]:
predicted_windows = prediction_denormalized[:4].copy()
predicted_windows

array([0., 1., 0., 0.])

In [66]:
# Simulazione per raggiungere i target
def simulate_adjustments(input_values):
    current_windows = input_values['window1_now'], input_values['window2_now'], input_values['window3_now'], input_values['window4_now']
    current_shutters = input_values['shutter1_now'], input_values['shutter2_now'], input_values['shutter3_now'], input_values['shutter4_now']
    
    predicted_windows = prediction_denormalized[:4].copy()
    predicted_shutters = prediction_denormalized[4:].copy()
    
    adjustments = {}
    
    
    for i, (predicted, current) in enumerate(zip(predicted_windows, current_windows)):
        if int(predicted) - current == 0:
            pass
        elif int(predicted) - current == 1:
            adjustments[f'window{i+1}'] = "Chiudi la finestra " + str(i+1)
        else:
            adjustments[f'window{i+1}'] = "Apri la finestra " + str(i+1)
    
    for i, (predicted, current) in enumerate(zip(predicted_shutters, current_shutters)):
        if abs(predicted-current) < 0.2:
            pass
        elif predicted > current:
            adjustments[f'shutter{i+1}'] = "Abbasso la tapparella " + str(i+1) + " del " + str(int((predicted-current)*100)) + "%"
        else:
            adjustments[f'shutter{i+1}'] = "Alzo la tapparella " + str(i+1) + " del " + str(int((current-predicted)*100)) + "%"
    
    return adjustments


In [67]:
# Simulare le modifiche necessarie per raggiungere i target
adjustments = simulate_adjustments(input_values)
print("Adjustments needed to reach target values:", adjustments)

Adjustments needed to reach target values: {'window2': 'Chiudi la finestra 2', 'shutter4': 'Abbasso la tapparella 4 del 90%'}


In [68]:
# Funzione per stampare le istruzioni per l'utente
def print_instructions(adjustments):
    instructions = []
    for key, value in adjustments.items():
        instructions.append(value)
    return instructions

In [69]:
# Stampare le istruzioni
instructions = print_instructions(adjustments)
for instruction in instructions:
    print(instruction)

Chiudi la finestra 2
Abbasso la tapparella 4 del 90%
